In [ ]:
# 1 - 1
import numpy as np


result = np.array([line.strip().split() for line in lines], dtype=int)
result = result.T
result = np.sort(result, axis=1)
result = np.sum(np.abs(result[0] - result[1]))
result

In [ ]:
# 1 - 2
from collections import defaultdict


result = 0
occurences = defaultdict(int)

x_list, y_list = map(list, zip(*[line.strip().split() for line in lines]))

for y in y_list:
    occurences[y] += 1
for x in x_list:
    result += int(x) * occurences[x]

result

In [ ]:
# 2 - 1


def test_order(numbers):
    return numbers == sorted(numbers) or numbers == sorted(numbers, reverse=True)


def test_difference(numbers):
    for i in range(1, len(numbers)):
        diff = abs(numbers[i - 1] - numbers[i])
        if diff not in [1, 2, 3]:
            return False
    return True


safe = 0
with open("02.txt", "r") as f:
    for line in f.readlines():
        numbers = line.strip().split()
        numbers = list(map(int, numbers))

        if test_difference(numbers) and test_order(numbers):
            safe += 1

safe

In [ ]:
# 2 - 2
from copy import copy


safe = 0
with open("02.txt", "r") as f:
    for line in f.readlines():
        numbers = line.strip().split()
        numbers = list(map(int, numbers))

        if test_difference(numbers) and test_order(numbers):
            safe += 1
        # test subsets
        else:
            for i in range(len(numbers)):
                reduced_list = copy(numbers)
                reduced_list.pop(i)
                if test_difference(reduced_list) and test_order(reduced_list):
                    safe += 1
                    break

safe

In [ ]:
# 3 - 1
import regex as re


with open("03.txt", "r") as f:
    memory = f.readlines()
    memory = "".join(memory)


def compute_memory(memory):
    pattern = re.compile(r"mul\((\d+),(\d+)\)")
    matches = pattern.findall(memory)
    result = 0
    for match in matches:
        result += int(match[0]) * int(match[1])

    return result


compute_memory(memory)

In [ ]:
# 3 - 2

memory2 = "do()" + memory
memory2 = memory2.split("do()")
memory2 = [m.split("don't()")[0] for m in memory2]

memory2 = "".join(memory2)
compute_memory(memory2)

In [ ]:
# 4 - 1
import numpy as np


with open("04.txt", "r") as f:
    puzzle = f.readlines()
    puzzle = [letter.strip() for letter in puzzle]
    puzzle = np.array([list(letter) for letter in puzzle])


def find_horizontal(puzzle, searched_word):
    total = 0
    for row in puzzle:
        word = "".join(row)
        total += word.count(searched_word) + word.count(searched_word[::-1])
    return total


def find_diagonal(puzzle, searched_word):
    total = 0
    for i in range(-len(puzzle), len(puzzle)):
        word = "".join(puzzle.diagonal(i))
        total += word.count(searched_word) + word.count(searched_word[::-1])
    return total


(
    find_horizontal(puzzle, "XMAS")
    + find_horizontal(puzzle.T, "XMAS")
    + find_diagonal(puzzle, "XMAS")
    + find_diagonal(np.fliplr(puzzle), "XMAS")
)

In [ ]:
# 4 - 2

word = "MAS"
result = 0

for i in range(len(puzzle) - len(word) + 1):
    for j in range(len(puzzle) - len(word) + 1):
        window = puzzle[i : i + len(word), j : j + len(word)]

        if find_diagonal(window, word) + find_diagonal(np.fliplr(window), word) == 2:
            result += 1

result

In [ ]:
# 5 - 1
from collections import defaultdict


rules = defaultdict(list)


def add_rule(before, after):
    rules[before] += [after]


def evaluate_rules(list):
    for i, before in enumerate(list):
        for after in list[i + 1 :]:
            if after not in rules[before]:
                return 0

    return list[len(list) // 2]


total = 0
with open("05.txt", "r") as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
    for line in lines:
        if "|" in line:
            add_rule(line.split("|")[0], line.split("|")[1])
        else:
            total += int(evaluate_rules(line.split(",")))
total

In [ ]:
# 5 - 2


def fix_rules(list, depth):
    for i, before in enumerate(list):
        for j, after in enumerate(list[i + 1 :]):
            real_j_position = j + i + 1
            # if rules are broken, swap and try again
            if after not in rules[before]:
                list[i], list[real_j_position] = list[real_j_position], list[i]
                return fix_rules(list, depth + 1)

    return list[len(list) // 2]


total = 0

for line in lines:
    if "|" in line:
        add_rule(line.split("|")[0], line.split("|")[1])
    else:
        if evaluate_rules(line.split(",")) == 0:
            total += int(fix_rules(line.split(","), depth=0))

total

In [ ]:
# 6 - 1
from collections import defaultdict
import numpy as np


def in_map(map, pos):
    return 0 <= pos[0] < map.shape[0] and 0 <= pos[1] < map.shape[1]


def obstacle(map, pos):
    if not in_map(map, pos):
        return False
    return map[pos[0], pos[1]] == "#"


def rotate90(direction):
    return np.array([direction[1], -direction[0]])


with open("06.txt", "r") as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
    map = np.array([list(line) for line in lines])

In [ ]:
# 6 - 2


def is_loop(map, agent_position, agent_direction, obstacle_position):
    if not in_map(map, obstacle_position):
        return False

    map[obstacle_position[0], obstacle_position[1]] = "#"
    visited_positions = defaultdict(tuple)

    while in_map(map, agent_position):
        # compute next possible position
        next_position = agent_position + agent_direction
        while obstacle(map, next_position):
            agent_direction = rotate90(agent_direction)
            next_position = agent_position + agent_direction

        # move
        agent_position = next_position

        # check for loop
        if visited_positions.get(tuple(agent_position)) == tuple(agent_direction):
            return True
        visited_positions[tuple(agent_position)] = tuple(agent_direction)

    return False


def agent_run(map, agent_position, agent_direction):
    obstacles = set()
    while in_map(map, agent_position):
        # compute next possible position
        next_position = agent_position + agent_direction
        while obstacle(map, next_position):
            agent_direction = rotate90(agent_direction)
            next_position = agent_position + agent_direction

        # mark current position with X
        map[agent_position[0], agent_position[1]] = "X"

        # try to place obstacle
        if not np.equal(next_position, STARTING_POSITION).all():
            if is_loop(
                map.copy(), STARTING_POSITION, STARTING_DIRECTION, next_position
            ):
                obstacles.add(tuple(next_position))

        # move
        agent_position = next_position

    return map, len(obstacles)


with open("06.txt", "r") as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
    corridor_map = np.array([list(line) for line in lines])

# agent starts at ^ symbol and faces up
agent_position = np.argwhere(corridor_map == "^")[0]
agent_direction = np.array([-1, 0])

STARTING_POSITION = agent_position.copy()
STARTING_DIRECTION = agent_direction.copy()

corridor_map, obstacles = agent_run(corridor_map, agent_position, agent_direction)

print("\n".join(["".join(row) for row in corridor_map]))
print("Number of steps: ", np.sum(corridor_map == "X"))
print("Number of obstacles: ", obstacles)

In [ ]:
# 7 - 1
import numpy as np
from tqdm import tqdm


def apply_mask(mask, numbers):
    result = 0

    for i in range(len(numbers)):
        if mask[i] == "2":
            result = int(str(result) + str(numbers[i]))
        elif mask[i] == "1":
            result += numbers[i]
        elif mask[i] == "0":
            result *= numbers[i]

    return result


def compute_operators(result, numbers, base=2):
    # get all variations true false of length numbers
    variations = []
    for i in range(base ** len(numbers)):
        operation_types = np.base_repr(i, base=base).zfill(len(numbers))
        variations.append(list(operation_types))

    for variation in variations:
        if apply_mask(variation, numbers) == result:
            return True
    return False


sum = 0

with open("07.txt", "r") as f:
    for line in tqdm(f.readlines()):
        data = line.split(":")
        result = int(data[0])
        numbers = data[1].strip().split(" ")
        numbers = list(map(int, numbers))

        if compute_operators(result, numbers):
            sum += result

sum

In [ ]:
# 7 - 2
sum = 0

with open("07.txt", "r") as f:
    for line in tqdm(f.readlines()):
        data = line.split(":")
        result = int(data[0])
        numbers = data[1].strip().split(" ")
        numbers = list(map(int, numbers))

        if compute_operators(result, numbers, base=3):
            sum += result

sum

In [ ]:
# 8 - 1
import numpy as np


def in_map(map, pos):
    return 0 <= pos[0] < map.shape[0] and 0 <= pos[1] < map.shape[1]


def get_frequency_location(map, i, j, draw_line):
    frequencies = []
    freq_type = map[i, j]
    initial_position = np.array([i, j])

    for x in range(0, map.shape[0]):
        for y in range(0, map.shape[1]):
            pair_position = np.array([x, y])
            if (
                map[x, y] == freq_type
                and not np.equal(initial_position, pair_position).all()
            ):
                direction = pair_position - initial_position

                # draw line connecting two frequencies
                if draw_line:
                    gcd = np.gcd.reduce(direction)
                    direction = direction // gcd

                    freq_position = initial_position + direction
                    while in_map(map, freq_position):
                        frequencies.append(freq_position.copy())
                        freq_position += direction

                    freq_position = initial_position - direction
                    while in_map(map, freq_position):
                        frequencies.append(freq_position.copy())
                        freq_position -= direction

                # get one frequence point
                else:
                    freq_position = initial_position - direction
                    if in_map(map, freq_position):
                        frequencies.append(freq_position.copy())

    return frequencies


def mark_frequencies(map, draw_line=False):
    frequency_map = map.copy()
    for i in range(map.shape[0]):
        for j in range(map.shape[1]):
            if map[i, j] == ".":
                continue
            frequencies = get_frequency_location(map, i, j, draw_line=draw_line)
            for freq in frequencies:
                frequency_map[freq[0], freq[1]] = "#"

    print("\n".join(["".join(row) for row in frequency_map]))
    return frequency_map


with open("08.txt", "r") as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
    map = np.array([list(line) for line in lines])

frequency_map = mark_frequencies(map)
np.sum(frequency_map == "#")

In [ ]:
# 8 - 2

frequency_map = mark_frequencies(map, draw_line=True)
np.sum(frequency_map == "#")

In [ ]:
# 9 - 1


def load_disk(disk):
    visualized_disk = []
    for i, slot in enumerate(disk):
        if i % 2 == 0:  # occupied slot
            visualized_disk += [str(i // 2)] * int(slot)
        else:  # empty slot
            visualized_disk += ["."] * int(slot)
    return visualized_disk


def get_hash(disk):
    res = 0
    for i, slot in enumerate(disk):
        if slot == ".":
            continue
        res += i * int(slot)
    return res


with open("09.txt", "r") as f:
    disk = f.readlines()[0].strip()

disk = load_disk(disk)

# get amount of free space
empty_slots = len([slot for slot in disk if slot == "."])

# defragment
for i in range(empty_slots):
    disk, slot = disk[:-1], disk[-1]
    try:
        disk[disk.index(".")] = slot
    except ValueError:  # there is no more slots to move
        break

get_hash(disk)

In [ ]:
# 9 - 2


def defragment_by_block(disk):
    number_of_ids = int(disk[-1])
    slot_start = len(disk) - 1

    for i in reversed(range(number_of_ids + 1)):
        # find the location and size of block we want to move
        slot_start = disk.index(str(i), 0, slot_start)
        slot_end = slot_start
        while slot_end < len(disk) and disk[slot_end] == disk[slot_start]:
            slot_end += 1

        block_length = slot_end - slot_start

        # find if there is a free sequence which can fit block
        pos, size = 0, 0
        last_filled_segment = 0
        while pos < slot_start:
            if disk[pos] == ".":
                size += 1
                if size == block_length:  # swap segments
                    disk[last_filled_segment + 1 : pos + 1] = [str(i)] * block_length
                    disk[slot_start:slot_end] = ["."] * block_length
                    break
            else:
                last_filled_segment = pos
                size = 0
            pos += 1
    return disk


with open("09.txt", "r") as f:
    disk = f.readlines()[0].strip()

disk = load_disk(disk)
disk = defragment_by_block(disk)

get_hash(disk)

In [ ]:
# 10 - 1


def hike(map, x, y, level=0):
    if x < 0 or y < 0 or x >= len(map) or y >= len(map[x]):
        return []
    if map[x][y] != level:
        return []
    if level == 9:
        return [(x, y)]

    return (
        hike(map, x - 1, y, level + 1)
        + hike(map, x + 1, y, level + 1)
        + hike(map, x, y - 1, level + 1)
        + hike(map, x, y + 1, level + 1)
    )


def count_trails(map, count_unique_paths):
    trails = 0
    for x in range(len(map)):
        for y in range(len(map[x])):
            if map[x][y] == 0:
                new_trails = hike(map, x, y)
                if not count_unique_paths:
                    new_trails = set(new_trails)
                trails += len(new_trails)

    return trails


with open("10.txt", "r") as f:
    map = f.readlines()
    map = [list(line.strip()) for line in map]
    map = [[int(x) for x in line] for line in map]

count_trails(map, count_unique_paths=False)

In [ ]:
# 10 - 2
count_trails(map, count_unique_paths=True)

In [ ]:
# 11 - 1
import functools
from tqdm import tqdm


@functools.cache
def blink(stone, number_of_blinks):
    if number_of_blinks == 0:
        return 1
    if stone == 0:
        return blink(1, number_of_blinks - 1)
    elif len(str(stone)) % 2 == 0:
        split_point = len(str(stone)) // 2
        left_stone = int(str(stone)[:split_point])
        right_stone = int(str(stone)[split_point:])
        return blink(left_stone, number_of_blinks - 1) + blink(
            right_stone, number_of_blinks - 1
        )
    return blink(stone * 2024, number_of_blinks - 1)


with open("11.txt", "r") as f:
    stones = f.readlines()[0].strip()

result = 0
for stone in tqdm(stones.split()):
    result += blink(int(stone), 25)
result

In [ ]:
# 11 - 2
result = 0
for stone in tqdm(stones.split()):
    result += blink(int(stone), 75)
result

In [ ]:
# 12 - 1
import numpy as np


def flood_fill(garden, crop, pos, value):
    visited = []
    stack = [pos]
    while stack:
        current_pos = stack.pop()
        for direction in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            try:
                new_pos = (current_pos[0] + direction[0], current_pos[1] + direction[1])
                if garden[new_pos[0], new_pos[1]] == crop:
                    garden[new_pos[0], new_pos[1]] = value
                    stack.append(new_pos)
                    visited.append(new_pos)
            except IndexError:
                continue
    return garden, visited


def get_number_of_neighbors(mask, tile):
    neighbors = 0
    for direction in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        try:
            if mask[tile[0] + direction[0], tile[1] + direction[1]]:
                neighbors += 1
        except IndexError:
            continue
    return neighbors


def compute_fence_cost(mask):
    total_perimeter = 0
    for tile in np.argwhere(mask):
        total_perimeter += 4 - get_number_of_neighbors(mask, tile)
    return np.sum(mask) * total_perimeter


with open("12.txt", "r") as f:
    garden = f.readlines()
    garden = np.array([list(line.strip()) for line in garden])
    garden = garden.astype(str)

cost = 0

crop_types = set(garden.flatten())
for crop in crop_types:
    visited = []
    for tile in np.argwhere(garden == crop):
        if tuple(tile) not in visited:
            segment, newly_visited = flood_fill(garden, crop, tile, "X")

            mask = np.array(garden == "X")
            cost += compute_fence_cost(mask)

            visited += newly_visited

            print(visited)

cost

In [ ]:
import numpy as np


DIRECTIONS = np.array([(0, 1), (0, -1), (1, 0), (-1, 0)])


def get_edges(garden, pos, crop):
    # get 3x3 window of the area
    mask = garden[pos[0] - 1 : pos[0] + 2, pos[1] - 1 : pos[1] + 2]
    mask = mask == crop
    mask.astype(int)
    # set convolution filters which detect edges
    edges = 0
    edge_filter = [
        np.array([[0, 1, -1], [0, 1, 1], [0, 0, 0]]),
        np.array([[0, 0, 0], [-1, 3, 0], [0, -1, 0]]),
    ]
    # compute for every rotation
    for edge in edge_filter:
        for rotation in range(4):
            mask = np.rot90(mask)
            if np.sum(mask * edge) == 3:
                edges += 1
    return edges


def get_perimeter(garden, pos, crop):
    neighbors = 0

    for direction in DIRECTIONS:
        new_pos = pos + direction

        if garden[tuple(new_pos)] == crop:
            neighbors += 1
    return 4 - neighbors


def get_area_cost(mask, garden, crop, pos, discount):
    area, cost = 0, 0
    mask[pos[0], pos[1]] = False
    stack = [pos]

    while stack:
        # increase fence cost by newly visited position
        current_pos = stack.pop()
        area += 1
        if discount:
            cost += get_edges(garden, current_pos, crop)
        else:
            cost += get_perimeter(garden, current_pos, crop)

        # try to find neighborhood areas
        for direction in DIRECTIONS:
            new_pos = current_pos + direction

            if garden[tuple(new_pos)] == crop and mask[tuple(new_pos)]:
                # found new unvisited neighbor
                mask[tuple(new_pos)] = False
                stack.append(new_pos)

    return mask, area * cost


def compute_fence_price(garden, discount=False):
    # create a mask to track visited areas
    mask = np.array(garden != None)

    # pad garden with "."
    garden = np.pad(garden, 1, "constant", constant_values=".")
    mask = np.pad(mask, 1, "constant", constant_values=False)

    total_cost = 0
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i, j]:
                crop = garden[i, j]
                mask, cost = get_area_cost(
                    mask, garden, crop, (i, j), discount=discount
                )
                total_cost += cost

    return total_cost


with open("12.txt", "r") as f:
    garden = f.readlines()
    garden = np.array([list(line.strip()) for line in garden])

compute_fence_price(garden)

In [ ]:
# 12 - 2

compute_fence_price(garden, discount=True)